<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_TensorFlow/blob/main/BidirectionalLSTMRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dataset = tfds.load('imdb_reviews', as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

batch_size = 32

train_dataset = train_dataset.shuffle(10000).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [3]:
example, label = next(iter(train_dataset))
print('Text:\n', example.numpy()[0])
print('\nLabel: ', label.numpy()[0])

Text:
 b'Match 1: Tag Team Table Match Bubba Ray and Spike Dudley vs Eddie Guerrero and Chris Benoit Bubba Ray and Spike Dudley started things off with a Tag Team Table Match against Eddie Guerrero and Chris Benoit. According to the rules of the match, both opponents have to go through tables in order to get the win. Benoit and Guerrero heated up early on by taking turns hammering first Spike and then Bubba Ray. A German suplex by Benoit to Bubba took the wind out of the Dudley brother. Spike tried to help his brother, but the referee restrained him while Benoit and Guerrero ganged up on him in the corner. With Benoit stomping away on Bubba, Guerrero set up a table outside. Spike dashed into the ring and somersaulted over the top rope onto Guerrero on the outside! After recovering and taking care of Spike, Guerrero slipped a table into the ring and helped the Wolverine set it up. The tandem then set up for a double superplex from the middle rope which would have put Bubba through the t

In [4]:
vectorize_layer = tf.keras.layers.TextVectorization(
    output_mode='int', output_sequence_length=100)

vectorize_layer.adapt(train_dataset.map(lambda x, y: x))

In [7]:
class CustomSentimentModel(tf.keras.Model):
    def __init__(self, vectorize_layer_instance, vocab_size, embedding_dim=64, **kwargs):
        super(CustomSentimentModel, self).__init__(**kwargs)
        self.vectorize_layer = vectorize_layer_instance
        self.embedding_layer = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)
        self.bilstm1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))
        self.bilstm2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')

        # Initialize custom weights and biases, which will be built in the `build` method
        self.output_kernel = None
        self.output_bias = None

    def build(self, input_shape):
        # The output of self.dense1 will be (batch_size, 64)
        # So the input dimension to the final custom output layer is 64
        self.output_kernel = self.add_weight(
            name='output_kernel',
            shape=(64, 1), # Input from dense1 is 64, output is 1
            initializer='glorot_uniform',
            trainable=True
        )
        self.output_bias = self.add_weight(
            name='output_bias',
            shape=(1,),
            initializer='zeros',
            trainable=True
        )
        super(CustomSentimentModel, self).build(input_shape)

    def call(self, inputs):
        x = self.vectorize_layer(inputs)
        x = self.embedding_layer(x)
        x = self.bilstm1(x)
        x = self.bilstm2(x)
        x = self.dense1(x)
        # Apply the custom final output weights and bias
        return tf.matmul(x, self.output_kernel) + self.output_bias

# Instantiate the custom model
vocab_size = len(vectorize_layer.get_vocabulary())
model = CustomSentimentModel(vectorize_layer_instance=vectorize_layer, vocab_size=vocab_size)


In [5]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.build(input_shape=(None,))


model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 64)        │     7,801,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 128)       │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,912,705 (30.18 MB)

 Trainable params: 7,912,705 (30.18 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(
    train_dataset,
    epochs=3,
    validation_data=test_dataset,
)

Epoch 1/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 321s 396ms/step - accuracy: 0.6927 - loss: 0.5312 - val_accuracy: 0.8147 - val_loss: 0.3974
Epoch 2/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 308s 393ms/step - accuracy: 0.9198 - loss: 0.1993 - val_accuracy: 0.7734 - val_loss: 0.4725
Epoch 3/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 320s 390ms/step - accuracy: 0.9763 - loss: 0.0675 - val_accuracy: 0.7895 - val_loss: 0.8240
